# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *
from regression import *
from utils import *

In [3]:
# cost_estimation_function = estimate_costs
cost_estimation_function = estimate_amortized_hardware_costs
compute_percentile_threshold = 75  # select systems >= this percentile of training compute
results_dir = 'results/amortized-75th/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_percentile_threshold=compute_percentile_threshold
)

In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(76, 5509, 94)

# Cost estimation

In [6]:
cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

==== System: Gemini Ultra ====
Trying Google TPU v4
Could not find hardware model for Google TPU v4


==== System: Qwen-72B ====
Could not find hardware model for Qwen-72B


==== System: Inflection-2 ====
Trying NVIDIA H100 SXM5
Found price: 39546.976

==== System: Yi-34B ====
Could not find hardware model for Yi-34B


==== System: ChatGLM3 ====
Could not find hardware model for ChatGLM3


==== System: Falcon 180B ====
Trying NVIDIA A100 SXM4 40 GB
Found price: 24875.0

==== System: Llama 2-70B ====
Trying NVIDIA A100 SXM4 80 GB
Found price: 25641.25

==== System: Llama 2-34B ====
Trying NVIDIA A100 SXM4 80 GB
Found price: 25641.25

==== System: Claude 2 ====
Could not find hardware model for Claude 2


==== System: xTrimoPGLM -100B ====
Trying NVIDIA A100 SXM4 40 GB
Found price: 24875.0

==== System: PaLM 2 ====
Trying Google TPU v4
Could not find hardware model for Google TPU v4


==== System: PanGu-Σ ====
Trying Huawei Ascend 910
Could not find hardware model for Huawei Ascend 910



In [7]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Batch size notes,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Cost
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,252.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,Industry,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN,NaN
46,Qwen-72B,Language,"Chat,Code generation","Jinze Bai, Shuai Bai, Yunfei Chu, Zeyu Cui, Ka...",SOTA improvement,"SOTA on several Chinese benchmarks, with highe...",Permissive license (depr.),https://huggingface.co/Qwen/Qwen-72B,NaN,NaN,...,Table 1 https://arxiv.org/abs/2309.16609\n(thi...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,API access,https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,NaN,Industry,checked,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,Yi-34B,Language,Chat,NaN,Significant use,2nd most popular model on HuggingFace: https:/...,Permissive license (depr.),https://arxiv.org/abs/2403.04652,NaN,NaN,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,ChatGLM3,Multimodal,"Chat,Visual question answering",NaN,SOTA improvement,"Aiming at GPT-4V, ChatGLM3 has implemented ite...",NaN,https://www.zhipuai.cn/en/news/76,NaN,Zhipu AI launches third-generation base model,...,NaN,Industry,checked,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,Xception,Vision,Image classification,François Chollet,Highly cited,NaN,NaN,https://arxiv.org/abs/1610.02357,11338.0,Xception: Deep Learning with Depthwise Separab...,...,NaN,Industry,NaN,NaN,NaN,43200.0,NaN,NaN,NaN,NaN
1095,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,6105.0,Google's Neural Machine Translation System: Br...,...,NaN,Industry,NaN,NaN,NaN,414720.0,NaN,NaN,NaN,NaN
1131,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14733.0,Mastering the game of Go with deep neural netw...,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2741.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,Industry,NaN,NaN,NaN,301.0,NaN,NaN,NaN,NaN


In [8]:
cost_df['Cost'].notna().sum()

14

Use the below to check data availability for specific systems

In [9]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['System'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [10]:
# TODO move to function

In [11]:
cost_df['Publication date']

39     2023-12-06
46     2023-11-30
54     2023-11-22
83     2023-11-02
92     2023-10-27
          ...    
1094   2016-10-07
1095   2016-09-26
1131   2016-01-27
1137   2015-12-08
1141   2015-10-01
Name: Publication date, Length: 76, dtype: datetime64[ns]

In [12]:
from_year_month = cost_df['Publication date'].apply(str)
cost_df['Publication date'] = from_year_month

In [13]:
cost_df['Publication date']

39      2023-12-06 00:00:00
46      2023-11-30 00:00:00
54      2023-11-22 00:00:00
83      2023-11-02 00:00:00
92      2023-10-27 00:00:00
               ...         
1094    2016-10-07 00:00:00
1095    2016-09-26 00:00:00
1131    2016-01-27 00:00:00
1137    2015-12-08 00:00:00
1141    2015-10-01 00:00:00
Name: Publication date, Length: 76, dtype: object

In [14]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [15]:
cost_df['Cost (inflation-adjusted)'].dropna()

135    1.684096e+07
174    2.118069e+06
186    2.858053e+06
263    5.449438e+07
279    9.809609e+05
334    1.072482e+06
375    1.066945e+06
439    8.047873e+05
472    2.027732e+05
542    3.469823e+06
549    6.438386e+05
724    3.089796e+06
810    1.479869e+05
835    1.095963e+05
Name: Cost (inflation-adjusted), dtype: float64

In [16]:
cost_df['Cost (inflation-adjusted)'].notna().sum()

14

# Regression

In [17]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [18]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

/Users/bencottier/miniconda3/envs/epoch/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.390
Model:                            OLS   Adj. R-squared:                  0.339
Method:                 Least Squares   F-statistic:                     7.657
Date:                Mon, 25 Mar 2024   Prob (F-statistic):             0.0171
Time:                        17:21:24   Log-Likelihood:                -11.756
No. Observations:                  14   AIC:                             27.51
Df Residuals:                      12   BIC:                             28.79
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -666.6113    243.119     -2.742      0.018   -1196.321    -136.901
x1             0.3327      0.120      2.767      0.017       0.071       0.595
==============================================================================
Omnibus:                        2.995   Durbin-Watson:                   2.423
Prob(Omnibus):                  0.224   Jarque-Bera (JB):                1.867
Skew:                           0.886   Prob(JB):                        0.393
Kurtosis:                       2.758   Cond. No.                     3.04e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.04e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
print_growth_rates(reg_results)

0.33 OOMs/year (95% CI: 0.07, 0.59)
2.2x/year (95% CI: 1.2x, 3.9x)
doubling time of 11 months (95% CI: 6, 51)


In [20]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [21]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,3.768991,0.874412,1.863811,5.674172,1.451938,6.086044,2015.00000,2015-01-01
1,3.802597,0.862481,1.923413,5.681781,1.506872,6.098322,2015.10101,2015-02-06
2,3.836202,0.850555,1.983002,5.689402,1.561698,6.110707,2015.20202,2015-03-15
3,3.869808,0.838635,2.042578,5.697038,1.616413,6.123203,2015.30303,2015-04-21
4,3.903413,0.826723,2.102140,5.704687,1.671014,6.135813,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,6.961518,0.335897,6.229661,7.693376,5.453329,8.469708,2024.59596,2024-08-06
96,6.995124,0.346590,6.239969,7.750278,5.475493,8.514755,2024.69697,2024-09-12
97,7.028729,0.357375,6.250075,7.807383,5.497285,8.560174,2024.79798,2024-10-19
98,7.062335,0.368245,6.259997,7.864673,5.518714,8.605956,2024.89899,2024-11-25


# Plots

In [22]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text='Cost of cloud compute to train large ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [23]:
label_systems = ['GNMT', 'Megatron-BERT', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini Ultra']

fig = px.scatter(
    cost_df.loc[cost_df['System'].isin(label_systems)],
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)

fig.add_scatter(
    x=cost_df['Publication date'],
    y=cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# legend on top
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='top',
    y=-0.15,
    xanchor='center',
    x=0.5,
))

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text='Cost of cloud compute to train large ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Export data

In [24]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Hosted access (no API),Gemini: A Family of Highly Capable Multimodal ...,2023-12-06 00:00:00,Google DeepMind,NaN,5.000000e+25,NaN,...,Google TPU v4,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,NaN,NaN
46,Qwen-72B,Language,"Chat,Code generation",Permissive license (depr.),NaN,2023-11-30 00:00:00,Alibaba,7.200000e+10,1.300000e+24,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,API access,Inflection-2: The Next Step Up,2023-11-22 00:00:00,Inflection AI,NaN,1.001000e+25,NaN,...,NVIDIA H100 SXM5,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,NaN,NaN
83,Yi-34B,Language,Chat,Permissive license (depr.),NaN,2023-11-02 00:00:00,01.AI,3.400000e+10,6.100000e+23,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,ChatGLM3,Multimodal,"Chat,Visual question answering",NaN,Zhipu AI launches third-generation base model,2023-10-27 00:00:00,Zhipu AI,1.300000e+11,1.092000e+24,1.050000e+12,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,Xception,Vision,Image classification,NaN,Xception: Deep Learning with Depthwise Separab...,2016-10-07 00:00:00,Google,2.285595e+07,4.360000e+20,3.500000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN
1095,GNMT,Language,Translation,NaN,Google's Neural Machine Translation System: Br...,2016-09-26 00:00:00,Google,2.780000e+08,6.900000e+21,3.600000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,96.0,NaN,NaN,NaN,NaN,NaN
1131,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27 00:00:00,DeepMind,NaN,1.900000e+21,2.940000e+07,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,NaN,Deep Speech 2: End-to-End Speech Recognition i...,2015-12-08 00:00:00,Baidu Research - Silicon Valley AI Lab,3.800000e+07,2.600000e+19,1.633392e+08,...,NVIDIA GTX Titan X,United States of America,NaN,NaN,NaN,0.45,NaN,NaN,NaN,NaN


In [25]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv', index=False)